In [ ]:
!pip install pytorch_lightning
!pip install torchmetrics
!pip install segmentation_models_pytorch
!pip install --upgrade timm
!pip install ttach
!pip install einops
!pip install rasterio

In [1]:
from model import BoundaryModel, EnsembleVotingModel
from data_loader import BoundaryData
from utils import free_memory

import pytorch_lightning as pl
import pandas as pd
import numpy as np
import torch
import os
import argparse
import datetime

In [2]:
# the experiments configurations
cfgs = [
    dict(
        backbone='selecsls42b',
        decoder='MANet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls42b',
        decoder='MANet',
        scheduler='',
        use_visible_only=False,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls42b',
        decoder='MANet',
        scheduler='',
        use_visible_only=True,
        include_extent=True,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls42b',
        decoder='fapn',
        scheduler='step',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls42b',
        decoder='manet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls42b',
        decoder='manet',
        scheduler='',
        use_visible_only=True,
        include_extent=True,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls60',
        decoder='MANet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls60',
        decoder='manet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='selecsls60b',
        decoder='MANet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='sehalonet33ts',
        decoder='MANet',
        scheduler='',
        use_visible_only=False,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='sebotnet33ts_256',
        decoder='MANet',
        scheduler='',
        use_visible_only=False,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='sebotnet33ts_256',
        decoder='MANet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='vgg13_bn',
        decoder='manet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='vgg13_bn',
        decoder='unet++',
        scheduler='onecycle',
        use_visible_only=False,
        include_extent=False,
        optimizer='adamw',
    ),
    dict(
        backbone='vgg19',
        decoder='manet',
        scheduler='',
        use_visible_only=True,
        include_extent=False,
        optimizer='sam',
    ),
    dict(
        backbone='vgg19_bn',
        decoder='unet++',
        scheduler='onecycle',
        use_visible_only=False,
        include_extent=False,
        optimizer='adamw',
    ),
    dict(
        backbone='vgg16',
        decoder='unet++',
        scheduler='onecycle',
        use_visible_only=False,
        include_extent=False,
        optimizer='adamw',
    ),
]

cfgs = pd.DataFrame(cfgs)
cfgs

,backbone,decoder,scheduler,use_visible_only,include_extent,optimizer
0,selecsls42b,MANet,,True,False,sam
1,selecsls42b,MANet,,False,False,sam
2,selecsls42b,MANet,,True,True,sam
3,selecsls42b,fapn,step,True,False,sam
4,selecsls42b,manet,,True,False,sam
5,selecsls42b,manet,,True,True,sam
6,selecsls60,MANet,,True,False,sam
7,selecsls60,manet,,True,False,sam
8,selecsls60b,MANet,,True,False,sam
9,sehalonet33ts,MANet,,False,False,sam


In [3]:
SEED = 417
log_dir = 'results'

In [4]:
# initial reproducibility
pl.seed_everything(SEED, workers=True)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"

Global seed set to 417


In [5]:
df_test = pd.read_csv('data/test.csv', converters = {'chip_id': str})
df_test

,chip_id,year,month
0,00,2021,3
1,00,2021,4
2,00,2021,8
3,00,2021,10
4,00,2021,11
...,...,...,...
73,12,2021,4
74,12,2021,8
75,12,2021,10
76,12,2021,11


In [6]:
checkpoint_paths = []

for index in cfgs.index:
    cfg = cfgs.iloc[index]
    
    args = dict(
        log_dir=log_dir,
#         num_classes=3,
        batch_size=4,
        seed=SEED,
        epochs=40,
#         patience=20,
        use_visible_bands_only=cfg['use_visible_only'],
        include_extent=cfg['include_extent'],
        include_background=False,
        include_distance=False,
        gpus='0',
        model=cfg['backbone'],
        decoder=cfg['decoder'],
        precision=32,
        optimizer=cfg['optimizer'],
        scheduler=cfg['scheduler'],
        warmup=10,
        weight_decay=1e-5,
        repeats=25,
        lr=1e-3,
    )


    args = argparse.Namespace(**args)
    
    date_time = datetime.datetime.now().strftime("%m%d-%H:%M")

    name = args.model + '_' + args.decoder
    version = name + '_' + date_time

    # make reproducible
    pl.seed_everything(args.seed, workers=True)

    print('\n')
    print(args)
    print('\n')
    
    # the datamodule here
    datamodule = BoundaryData(
        batch_size=args.batch_size, 
        seed=args.seed, 
        workers=8,
        repeats=args.repeats, 
        use_visible_bands_only=args.use_visible_bands_only,
        include_extent=args.include_extent,
        include_distance=args.include_distance,
        include_background=args.include_background,
        data_root='data/',
    )
    datamodule.setup()
    
    # call the model here
    model = BoundaryModel(args)

    # Initialize simple Callbacks # not optimal as it leads to 2 model saved
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
          verbose=False,
          save_last=True,
          save_weights_only=True
    )
    callbacks = [checkpoint_callback]

    # SET LOGGER
    tb_logger = pl.loggers.TensorBoardLogger(
        save_dir=args.log_dir,
        name=name,
        version=version, 
    )

    # Initialize a trainer
    trainer = pl.Trainer(
        callbacks=callbacks,
#         deterministic=True if (args.decoder != 'fapn') else None,
        accelerator='gpu',
        devices=[int(t) for t in args.gpus.split(',')],
        gradient_clip_val=1.0 if args.optimizer != 'sam' else None,
        gradient_clip_algorithm='norm',
        max_epochs=args.epochs,
        logger=tb_logger,
        num_sanity_val_steps=0,
        precision=args.precision,
    )

    # Train the model
    trainer.fit(model, datamodule)
    
    # include the paths in the list
    checkpoint_paths.append(f"{trainer.checkpoint_callback.dirpath}/last.ckpt")

In [7]:
checkpoint_paths

In [9]:
# clear space used up during training to give room for inferencing
free_memory([model, trainer, datamodule])

# Prediction

In [10]:
ss = pd.read_csv('SampleSubmission.csv')
ss.shape

(851968, 2)

In [11]:
os.makedirs('submissions', exist_ok=True)

In [12]:
all_predictions = []
dfs = []
for index in cfgs.index:
    cfg = cfgs.iloc[index]
    
    args = dict(
        log_dir=log_dir,
#         num_classes=3,
        batch_size=4,
        seed=SEED,
        epochs=40,
#         patience=20,
        use_visible_bands_only=cfg['use_visible_only'],
        include_extent=cfg['include_extent'],
        include_background=False,
        include_distance=False,
        gpus='0',
        model=cfg['backbone'],
        decoder=cfg['decoder'],
        precision=32,
        optimizer=cfg['optimizer'],
        scheduler=cfg['scheduler'],
        warmup=10,
        weight_decay=1e-5,
        repeats=25,
        lr=1e-3,
    )


    args = argparse.Namespace(**args)
    
    date_time = datetime.datetime.now().strftime("%m%d-%H:%M")

    name = args.model + '_' + args.decoder
    version = name + '_' + date_time

    # make reproducible
    pl.seed_everything(args.seed, workers=True)

    print('\n')
    print(args)
    print('\n')
    
    # the datamodule here
    datamodule = BoundaryData(
        batch_size=args.batch_size, 
        seed=args.seed, 
        workers=8,
        repeats=args.repeats, 
        use_visible_bands_only=args.use_visible_bands_only,
        include_extent=args.include_extent,
        include_distance=args.include_distance,
        include_background=args.include_background,
        data_root='data/',
    )
    datamodule.setup()
    
    # call the model here
    voting_model = EnsembleVotingModel(
        BoundaryModel,  # the model class to load trained models from
        [checkpoint_paths[index]], # list of all trained models
        use_tta=True,   # using test time augmentation
        tta_mode='max', # max merging of the test time augmentation used
        threshold=0.9,  # using a very high threshold for each model prediction before voting
        soft=True,     # using hard voting (mode suggested by the models)
    )

    # Initialize a trainer
    trainer = pl.Trainer(
        accelerator='gpu',
        devices=[int(t) for t in args.gpus.split(',')],
        precision=args.precision,
    )

    # predicting !!!!
    predictions = trainer.predict(voting_model, datamodule.test_dataloader())
    predictions = torch.cat(predictions, axis=0)  # combine the batches
    predictions = predictions[:, 0].transpose(1,2).flatten()
    
    sub_tta = ss[['tile_row_column']]
    sub_tta['label'] = predictions
    
    version = checkpoint_paths[index].split(os.sep)[-3]
    sub_tta.to_csv(f"submissions/{version}.csv", index=False)
    
    dfs.append(sub_tta)
    all_predictions.append(predictions)
    
    free_memory([voting_model, trainer, datamodule])

Global seed set to 417




Namespace(log_dir='results', batch_size=4, seed=417, epochs=40, use_visible_bands_only=True, include_extent=False, include_background=False, include_distance=False, gpus='0', model='selecsls42b', decoder='MANet', precision=32, optimizer='sam', scheduler='', warmup=10, weight_decay=1e-05, repeats=25, lr=0.001)


Using Timm encoder


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

Global seed set to 417


...

Predicting: 0it [00:00, ?it/s]

Global seed set to 417




Namespace(log_dir='results', batch_size=4, seed=417, epochs=40, use_visible_bands_only=False, include_extent=False, include_background=False, include_distance=False, gpus='0', model='vgg16', decoder='unet++', precision=32, optimizer='adamw', scheduler='onecycle', warmup=10, weight_decay=1e-05, repeats=25, lr=0.001)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

In [23]:
num = len(all_predictions)
preds = 1 * (torch.stack(all_predictions, axis=0).sum(axis=0) >= (num + 1)//2)

sub = ss[['tile_row_column']]
sub['label'] = preds

sub.to_csv('ens_sub.csv', index=False)
print(sub.label.value_counts())
sub.head()

0    789030
1     62938
Name: label, dtype: int64


,tile_row_column,label
0,Tile00_0_0,0
1,Tile00_0_1,0
2,Tile00_0_2,0
3,Tile00_0_3,0
4,Tile00_0_4,0
